In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
from  langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', google_api_key = os.getenv('GEMINI_KEY'))


In [9]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.graph.message import add_messages
from typing import Annotated, Literal, TypedDict
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AnyMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

In [3]:
def call_model(state:MessagesState):
    messages = state['messages']
    response = llm.invoke(messages)
    return {"messages":[response]}

In [5]:
workflow =StateGraph(MessagesState)
workflow.add_node('chatbot', call_model)
workflow.add_edge(START, 'chatbot')
workflow.add_edge('chatbot', END)
app = workflow.compile()

In [10]:
class MessagesState(TypedDict):
    messages:Annotated[list[AnyMessage], add_messages]

In [11]:
@tool
def search(query:str):
    """this is a custom tool """
    if 'sf' in query.lower() or "san fransisco" in query.lower():
        return 'its 60 degress and foggy'
    return 'its 90 degress and sunny '

In [12]:
tools = [search]

In [13]:
tool_node = ToolNode(tools)

In [14]:
llm_with_tools = llm.bind_tools(tools)

In [15]:
def call_model(state=MessagesState):
    messages = state['messages']
    resposne = llm_with_tools.invoke(messages)
    return {"messages":[resposne]}

In [18]:
resposne = call_model({'messages':["how is weather in sf"]})
resposne

{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'search', 'arguments': '{"query": "weather in San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-1922a785-0b2a-4162-b90a-9d7d088d7ff9-0', tool_calls=[{'name': 'search', 'args': {'query': 'weather in San Francisco'}, 'id': '7572b7f7-261b-4bc2-a028-08d89a506eff', 'type': 'tool_call'}], usage_metadata={'input_tokens': 13, 'output_tokens': 6, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}})]}

In [23]:
def router_function(state:MessagesState)->Literal["tools", END]:
    messages = state["messages"]
    last_messgaes = messages[-1]
    if last_messgaes.tool_calls:
        return "tools"
    return END

In [29]:
workflow = StateGraph(MessagesState)
workflow.add_node('agent', call_model)
workflow.add_node('tools', tool_node)
workflow.add_edge(START,"agent")
workflow.add_conditional_edges("agent", router_function, {"tools":"tools", END:END})


In [25]:
app.invoke({"messages":['What is the weather in sf']})

{'messages': [HumanMessage(content='What is the weather in sf', additional_kwargs={}, response_metadata={}, id='6a29eed8-b859-4b62-8f60-f05b0020c511'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search', 'arguments': '{"query": "weather in sf"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-8437365a-03cc-4bfd-a783-d7d7cc20ff0b-0', tool_calls=[{'name': 'search', 'args': {'query': 'weather in sf'}, 'id': 'f6423d5f-ce35-4cd9-a09a-c982b0a68de6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 14, 'output_tokens': 5, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='its 60 degress and foggy', name='search', id='6e8a4cec-c461-4760-9893-6c8c50a1d7cf', tool_call_id='f6423d5f-ce35-4cd9-a09a-c982b0a68de6')]}

In [30]:
workflow.add_edge("tools", 'agent')

In [31]:
app = workflow.compile()

In [35]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()


In [33]:
workflow = StateGraph(MessagesState)
workflow.add_node("agent",call_model)
workflow.add_node('tools', tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges('agent', router_function, {"tools":"tools", END:END})
workflow.add_edge("tools", 'agent')

In [36]:
app =  workflow.compile(checkpointer=memory)

In [37]:
config = {"configurable":{"thread_id":"1"}}

In [38]:
events =app.stream({"messages":["hi there my name is pavan"]}, config, stream_mode="values")

In [39]:
for event in events:
    event['messages'][-1].pretty_print()

================================ Human Message =================================

hi there my name is pavan
================================== Ai Message ==================================

Hello Pavan, nice to meet you! How can I help you today?


In [41]:
events =app.stream({"messages":["hi what is my name"]}, config, stream_mode="values")

In [42]:
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi what is my name
================================== Ai Message ==================================

Your name is Pavan.


In [43]:
memory.get(config)

{'v': 3,
 'ts': '2025-04-07T13:09:06.355609+00:00',
 'id': '1f013b17-c458-68f4-8004-60ccb0ac276f',
 'channel_versions': {'__start__': '00000000000000000000000000000005.0.7583310078978824',
  'messages': '00000000000000000000000000000006.0.21738359510664818',
  'branch:to:agent': '00000000000000000000000000000006.0.5194364469137897'},
 'versions_seen': {'__input__': {},
  '__start__': {'__start__': '00000000000000000000000000000004.0.3643336943363277'},
  'agent': {'branch:to:agent': '00000000000000000000000000000005.0.6617083645779094'}},
 'channel_values': {'messages': [HumanMessage(content='hi there my name is pavan', additional_kwargs={}, response_metadata={}, id='2eb98487-a192-4833-a3d9-1b5eb7f36f1d'),
   AIMessage(content='Hello Pavan, nice to meet you! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-51fb29cb-7